In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import os, glob

print("All model files I can see under MyDrive:\n")
for p in glob.glob("/content/drive/MyDrive/**/*", recursive=True):
    if p.endswith(".pth") or p.endswith(".pt") or p.endswith(".safetensors"):
        print(p)


Mounted at /content/drive
All model files I can see under MyDrive:

/content/drive/MyDrive/Deepfake/best_fusion_model.pth
/content/drive/MyDrive/Deepfake/deit_fusion_epoch5.pth
/content/drive/MyDrive/Deepfake/early_fusion_model/best_mlp_state.pth
/content/drive/MyDrive/Deepfake/early_fusion_model/early_fusion_mlp_final.pth


In [ ]:
# ---- Install packages (one time per session) ----
import subprocess, sys
subprocess.check_call([sys.executable, "-m", "pip", "install", "-q",
                       "transformers", "timm", "tqdm", "onnxruntime",
                       "gradio", "dlib", "opencv-python"])

# -------------------- Imports & config --------------------
import os, cv2, dlib
import numpy as np
from PIL import Image

import torch
import torch.nn as nn
import torch.nn.functional as F
from transformers import AutoImageProcessor, DeiTModel
import gradio as gr

IMG_SIZE     = 256
SYM_DIM      = 50
NUM_CLASSES  = 2
DEVICE       = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", DEVICE)

WEIGHT_PATH  = "/content/drive/MyDrive/Deepfake/deit_fusion_epoch5.pth"
LANDMARKS    = "shape_predictor_68_face_landmarks.dat"

# -------------------- Landmarks download --------------------
if not os.path.exists(LANDMARKS):
    print("Downloading landmark predictor...")
    os.system("wget -q http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2")
    os.system("bunzip2 -f shape_predictor_68_face_landmarks.dat.bz2")
    print("✓ Landmarks downloaded")

# -------------------- Symmetry extractor --------------------
class FacialSymmetryExtractor:
    def __init__(self, landmarks=LANDMARKS, dim=SYM_DIM):
        self.detector  = dlib.get_frontal_face_detector()
        self.predictor = dlib.shape_predictor(landmarks)
        self.dim       = dim

    def calc(self, pts):
        cx = np.mean(pts[:, 0])
        pairs = [
            (0,16),(1,15),(2,14),(3,13),(4,12),(5,11),(6,10),(7,9),
            (17,26),(18,25),(19,24),(20,23),(21,22),
            (36,45),(37,44),(38,43),(39,42),(40,47),(41,46),
            (31,35),(32,34),(48,54),(49,53),(50,52),(58,56),(59,55)
        ]
        feats = []
        for l, r in pairs:
            ld = abs(pts[l, 0] - cx)
            rd = abs(pts[r, 0] - cx)
            feats.extend([ld / (rd + 1e-6), abs(pts[l, 1] - pts[r, 1])])

        le, re = pts[36:42], pts[42:48]
        lw, rw = np.linalg.norm(le[3] - le[0]), np.linalg.norm(re[3] - re[0])
        lh, rh = np.linalg.norm(le[1] - le[5]), np.linalg.norm(re[1] - re[5])
        feats.extend([lw / (rw + 1e-6), lh / (rh + 1e-6)])

        if len(feats) < self.dim:
            feats.extend([0] * (self.dim - len(feats)))
        return np.array(feats[:self.dim], dtype=np.float32)

    def extract(self, arr):
        try:
            gray = cv2.cvtColor(arr, cv2.COLOR_RGB2GRAY)
        except:
            gray = cv2.cvtColor(arr, cv2.COLOR_BGR2GRAY)
        faces = self.detector(gray)
        if not faces:
            return np.zeros(self.dim, dtype=np.float32)
        largest = max(faces, key=lambda f: f.width() * f.height())
        lm = self.predictor(gray, largest)
        pts = np.array([[lm.part(i).x, lm.part(i).y] for i in range(68)], dtype=np.float32)
        return self.calc(pts)

sym_extractor = FacialSymmetryExtractor()
print("Symmetry extractor ready. SYM_DIM =", SYM_DIM)

# -------------------- Processor & model --------------------
processor = AutoImageProcessor.from_pretrained("facebook/deit-small-patch16-224")

class EarlyFusionHF(nn.Module):
    def __init__(self, deit_model, sym_dim=SYM_DIM, num_classes=NUM_CLASSES):
        super().__init__()
        self.deit = deit_model
        self.fc   = nn.Linear(384 + sym_dim, num_classes)  # DeiT-small CLS dim = 384

    def forward(self, x, sym):
        outputs = self.deit(x)
        cls_tok = outputs.last_hidden_state[:, 0, :]
        fused   = torch.cat([cls_tok, sym], dim=1)
        return self.fc(fused)

print("Loading DeiT backbone...")
deit_model = DeiTModel.from_pretrained("facebook/deit-small-patch16-224").to(DEVICE)
deit_model.eval()

model = EarlyFusionHF(deit_model, sym_dim=SYM_DIM, num_classes=NUM_CLASSES).to(DEVICE)

if not os.path.exists(WEIGHT_PATH):
    raise FileNotFoundError(f"Checkpoint not found: {WEIGHT_PATH}")

state = torch.load(WEIGHT_PATH, map_location=DEVICE)
model.load_state_dict(state)
model.eval()
print("✓ Trained model loaded from", WEIGHT_PATH)

GLOBAL_MODEL = model

# -------------------- Prediction logic --------------------
def predict_from_pil(pil_image):
    if pil_image is None:
        return "ERROR", 0.0, {"error": "No image"}
    try:
        img_rgb = np.array(pil_image.convert("RGB"))
        sym = sym_extractor.extract(img_rgb)
        sym_t = torch.tensor(sym, dtype=torch.float32).unsqueeze(0).to(DEVICE)

        inputs = processor(images=pil_image, return_tensors="pt")
        x = inputs["pixel_values"].to(DEVICE)

        with torch.no_grad():
            out   = GLOBAL_MODEL(x, sym_t)
            probs = F.softmax(out.squeeze(0), dim=0).cpu().numpy()
        pred  = int(out.argmax(1).item())
        label = "FAKE" if pred == 1 else "REAL"
        conf  = float(probs.max())

        sym_score = float(np.mean(np.abs(sym)))

        if label == "FAKE":
            reason = (
                "The model detected patterns consistent with manipulation, including "
                "irregular facial symmetry and atypical texture patterns."
            )
        else:
            reason = (
                "Facial symmetry and texture patterns look consistent with typical real images, "
                "and no strong manipulation cues were found."
            )

        details = {
            "confidence": conf,
            "symmetry_score": sym_score,
            "reason": reason,
        }
        return label, conf, details
    except Exception as e:
        return "ERROR", 0.0, {"error": str(e)}

# -------------------- Pair analysis helper --------------------
def analyse_pair(image_a, image_b):
    if image_a is None or image_b is None:
        return (
            gr.update(
                value="❌ Please upload both Image A and Image B.",
                visible=True,
            ),
            gr.update(value="", visible=False),
            gr.update(value="", visible=False),
        )

    label_a, conf_a, details_a = predict_from_pil(image_a)
    label_b, conf_b, details_b = predict_from_pil(image_b)

    if label_a == "ERROR":
        return (
            gr.update(value=f"Error on Image A: {details_a.get('error','Unknown')}", visible=True),
            gr.update(value="", visible=False),
            gr.update(value="", visible=False),
        )
    if label_b == "ERROR":
        return (
            gr.update(value=f"Error on Image B: {details_b.get('error','Unknown')}", visible=True),
            gr.update(value="", visible=False),
            gr.update(value="", visible=False),
        )

    conf_a_pct = details_a["confidence"] * 100.0
    conf_b_pct = details_b["confidence"] * 100.0

    combo = f"{label_a.upper()} + {label_b.upper()}"
    if label_a == "REAL" and label_b == "FAKE":
        combo_expl = "Image A looks authentic, while Image B shows strong manipulation cues."
    elif label_a == "FAKE" and label_b == "REAL":
        combo_expl = "Image A appears manipulated, while Image B looks authentic."
    elif label_a == "FAKE" and label_b == "FAKE":
        combo_expl = "Both images are likely deepfakes, possibly generated or edited independently."
    else:
        combo_expl = "Both images look real; no strong deepfake artefacts were detected."

    summary_md = f"""### 🧩 Pair summary

- **Combination:** `{combo}`
- **Explanation:** {combo_expl}
"""

    details_md = f"""### 🖼️ Image A

- **Prediction:** `{label_a}`
- **Confidence:** {conf_a_pct:.2f}%
- **Symmetry score:** {details_a.get("symmetry_score",0):.2f}

---

### 🖼️ Image B

- **Prediction:** `{label_b}`
- **Confidence:** {conf_b_pct:.2f}%
- **Symmetry score:** {details_b.get("symmetry_score",0):.2f}
"""

    return (
        gr.update(value="", visible=False),
        gr.update(value=summary_md, visible=True),
        gr.update(value=details_md, visible=True),
    )

# -------------------- Gradio UI --------------------
custom_css = """
.gradio-container {max-width: 1100px !important; font-family: 'Inter', system-ui, sans-serif;}
* {box-sizing: border-box;}
.main-card {
  background: #020617;
  border-radius: 26px;
  border: 1px solid rgba(148,163,184,0.45);
  padding: 28px 32px;
  box-shadow: 0 18px 45px rgba(15,23,42,0.75);
}
.section-title {
  font-size: 2.4rem;
  font-weight: 800;
  text-align: center;
  color: #c4b5fd;
  margin-bottom: 0.3rem;
}
.section-subtitle {
  text-align: center;
  font-size: 1.02rem;
  color:#9ca3af;
  margin-bottom: 2.2rem;
}
.chip {
  display:inline-flex;
  align-items:center;
  gap:8px;
  padding:4px 10px;
  border-radius:999px;
  background:#0f172a;
  border:1px solid rgba(148,163,184,0.5);
  color:#e5e7eb;
  font-size:0.78rem;
}
.pill-button {
  border-radius:999px !important;
  font-weight:600 !important;
  letter-spacing:0.02em;
  border:none !important;
  background: linear-gradient(90deg,#6366f1,#a855f7,#ec4899) !important;
  color:white !important;
  padding:12px 26px !important;
}
.pill-button:hover {
  box-shadow: 0 0 25px rgba(129,140,248,0.7);
  transform: translateY(-1px);
  transition: all 0.12s ease-out;
}
.secondary-pill {
  border-radius:999px !important;
  font-weight:500 !important;
  border:1px solid rgba(148,163,184,0.5) !important;
  background:rgba(15,23,42,0.9) !important;
  color:#e5e7eb !important;
}
.upload-box {
  border:2px dashed rgba(148,163,184,0.8) !important;
  border-radius:20px !important;
  background:rgba(15,23,42,0.8) !important;
}
.preview-panel {
  background: radial-gradient(circle at top, #0b1120, #020617);
  border-radius:20px;
  border:1px solid rgba(148,163,184,0.5);
  padding:10px;
}
.result-banner {
  border-radius:22px;
  padding:24px 26px;
  margin-bottom:18px;
}
.result-badge {
  display:inline-flex;
  align-items:center;
  justify-content:center;
  width:74px;
  height:74px;
  border-radius:22px;
  background:#0b1120;
  border:2px solid rgba(248,113,113,0.8);
  color:#fecaca;
  font-size:2rem;
}
.result-title {
  font-size:2.0rem;
  font-weight:800;
  letter-spacing:0.03em;
}
.result-sub {
  font-size:0.98rem;
  color:#e5e7eb;
}
.score-box {
  background:#020617;
  border-radius:16px;
  border:1px solid rgba(148,163,184,0.6);
  padding:14px 16px;
  font-size:0.9rem;
  color:#e5e7eb;
}
.score-label {
  color:#9ca3af;
  font-size:0.8rem;
  text-transform:uppercase;
  letter-spacing:0.06em;
}
.score-value {
  font-size:1.02rem;
  font-weight:600;
}
.prob-row {
  display:flex;
  justify-content:space-between;
  font-size:0.88rem;
}
.details-box {
  border-radius: 16px !important;
  border: 1px solid rgba(79,70,229,0.5) !important;
  background: radial-gradient(circle at top left, rgba(79,70,229,0.2), rgba(15,23,42,1)) !important;
  padding: 1.1rem 1.2rem !important;
  color:#e5e7eb;
}
.error-box {color:#fecaca;}

/* Glow around compare images */
.main-card .gradio-image {
  border-radius: 18px !important;
  box-shadow: 0 0 25px rgba(56,189,248,0.25);
}
.main-card .gradio-image:hover {
  box-shadow: 0 0 35px rgba(129,140,248,0.45);
  transition: box-shadow 0.12s ease-out;
}
"""

def go_tab(idx):  # 0:Home 1:Upload 2:Compare 3:Result 4:About
    return gr.Tabs(selected=idx)

def handle_upload(filepath):
    if filepath is None:
        return (gr.update(value="❌ Please upload a JPG or PNG image.", visible=True),
                None,
                gr.update(interactive=False),
                gr.update(visible=False),
                "")
    try:
        pilimg = Image.open(filepath).convert("RGB")
        info = f"Size: {pilimg.width}×{pilimg.height} | Format: {pilimg.format or 'Unknown'}"
        return (
            gr.update(value="", visible=False),
            pilimg,
            gr.update(interactive=True),
            gr.update(visible=True),
            info,
        )
    except Exception as e:
        return (gr.update(value=f"❌ Error: {e}", visible=True),
                None,
                gr.update(interactive=False),
                gr.update(visible=False),
                "")

def analyse_and_go(image):
    if image is None:
        return ("Please upload an image first.", "", "", "", go_tab(1))

    label, conf, details = predict_from_pil(image)
    if label == "ERROR":
        return (f"Processing error: {details.get('error','Unknown')}", "", "", "", go_tab(1))

    is_fake = (label == "FAKE")
    conf_pct = conf * 100.0
    sym = details.get("symmetry_score", 0.0)
    reason = details.get("reason", "")

    if is_fake:
        banner_color = "rgba(127,29,29,0.85)"
        text_color   = "#fecaca"
        title_text   = "DEEPFAKE DETECTED"
        sub_text     = "This image appears to be AI‑generated or manipulated."
        icon         = "⛔"
    else:
        banner_color = "rgba(22,101,52,0.90)"
        text_color   = "#bbf7d0"
        title_text   = "LIKELY REAL"
        sub_text     = "No strong evidence of manipulation was detected."
        icon         = "✅"

    banner_html = f"""
    <div class="result-banner" style="background:{banner_color}; border:1px solid rgba(15,23,42,0.8);">
      <div style="display:flex; gap:20px; align-items:center;">
        <div class="result-badge">{icon}</div>
        <div>
          <div class="result-title" style="color:{text_color};">{title_text}</div>
          <div class="result-sub">{sub_text}</div>
        </div>
      </div>
    </div>
    """

    # Decide probabilities based on is_fake
    if is_fake:
        real_pct = (1 - conf) * 100
        fake_pct = conf_pct
    else:
        real_pct = conf_pct
        fake_pct = (1 - conf) * 100

    score_html = f"""
  <div style="display:flex; gap:16px; margin-bottom:14px;">
    <div class="score-box" style="flex:1;">
      <div class="score-label">Confidence</div>
      <div class="score-value">{conf_pct:.1f}%</div>
    </div>
    <div class="score-box" style="flex:1;">
      <div class="score-label">Probabilities</div>
      <div class="prob-row">
        <span>Real:</span><span>{real_pct:.1f}%</span>
      </div>
      <div class="prob-row">
        <span>Fake:</span><span>{fake_pct:.1f}%</span>
      </div>
    </div>
  </div>
  <div style="margin:10px 0 4px; font-size:0.86rem; color:#9ca3af;">
    Model: DeiT Vision Transformer + Facial Symmetry Features
  </div>
  <div style="width:100%; height:10px; border-radius:999px; background:#111827; overflow:hidden;">
    <div style="width:{conf_pct}%; height:100%;
                background:linear-gradient(90deg,#22c55e,#eab308,#f97316,#ef4444);"></div>
  </div>
  """


    explain_md = f"""
### 🧠 Why this result?

- **Decision:** `{label}` with **{conf_pct:.2f}%** confidence
- **Symmetry analysis:** average symmetry score **{sym:.2f}** (higher = more asymmetry)
- **Reasoning:** {reason}

> Note: This is an AI prediction. Always verify with multiple methods and human review.
"""

    return ("", banner_html, score_html, explain_md, go_tab(3))

# -------------------- Build tabs --------------------
with gr.Blocks(title="Deepfake Detection", css=custom_css) as demo:
    with gr.Tabs(selected=0) as tabs:

        # PAGE 1 – Home (centered hero + floating panel)
        with gr.Tab("Home", id=0):
            with gr.Column(elem_classes="main-card"):
                gr.HTML(
                    """
<div style="display:flex; flex-direction:column; align-items:center; text-align:center; gap:26px;">
  <div style="max-width:720px;">
    <div style="font-size:0.9rem; text-transform:uppercase; letter-spacing:0.24em;
                color:#a5b4fc; margin-bottom:10px;">
      🔒 DEEPFAKE DETECTION
    </div>
    <div style="font-size:2.7rem; font-weight:800; line-height:1.15; color:#e5e7eb;">
      Analyze face images for<br/>
      <span style="color:#c4b5fd;">AI‑generated manipulation</span>
    </div>
    <p style="margin-top:16px; font-size:0.98rem; color:#9ca3af;">
      Upload a face photo and this tool will estimate whether it is likely real or a deepfake,
      using transformer features and facial symmetry cues.
    </p>

    <div style="display:flex; flex-wrap:wrap; justify-content:center;
                gap:10px; margin:18px 0 8px;">
      <div class="chip">🎯 Deepfake probability score</div>
      <div class="chip">⚡ Results in under a second</div>
      <div class="chip">🧠 Symmetry‑aware transformer</div>
    </div>

    <div style="margin-top:18px; display:flex; justify-content:center;">
"""
                )
                start_btn = gr.Button("🚀 START ANALYZING", elem_classes="pill-button")
                gr.HTML(
                    """
    </div>

    <div style="margin-top:18px; font-size:0.82rem; color:#6b7280;">
      Designed for research labs, security teams, and AI enthusiasts exploring deepfake detection.
    </div>
  </div>

  <div style="flex:1.2; width:100%; max-width:720px; margin-top:24px;">
    <div style="
      position:relative;
      border-radius:24px;
      padding:18px 20px;
      border:1px solid rgba(148,163,184,0.55);
      background:linear-gradient(135deg, rgba(15,23,42,0.88), rgba(30,64,175,0.45));
      box-shadow:0 30px 60px rgba(15,23,42,0.9);
      backdrop-filter:blur(14px);
      -webkit-backdrop-filter:blur(14px);
      overflow:hidden;">

      <div style="
        position:absolute;
        inset:-40%;
        background:radial-gradient(circle at top left, rgba(94,234,212,0.28), transparent 60%);
        opacity:0.9;
        pointer-events:none;">
      </div>

      <div style="position:relative; z-index:1;">
        <div style="font-size:0.8rem; text-transform:uppercase;
                    letter-spacing:0.15em; color:#9ca3af;">
          Live detector snapshot
        </div>

        <div style="margin-top:10px; font-size:1.1rem; color:#e5e7eb; font-weight:600;">
          Your deepfake lab, at a glance
        </div>

        <div style="margin-top:14px; display:grid; grid-template-columns:1fr 1fr; gap:10px; font-size:0.82rem;">
          <div style="padding:10px 12px; border-radius:14px;
                      background:rgba(15,23,42,0.95); border:1px solid rgba(74,222,128,0.4);">
            <div style="color:#6ee7b7; font-size:0.75rem; text-transform:uppercase; letter-spacing:0.08em;">
              Detector mode
            </div>
            <div style="margin-top:4px; font-weight:600;">Face authenticity</div>
            <div style="margin-top:2px; color:#9ca3af;">Vision transformer + symmetry</div>
          </div>

          <div style="padding:10px 12px; border-radius:14px;
                      background:rgba(15,23,42,0.95); border:1px solid rgba(96,165,250,0.45);">
            <div style="color:#93c5fd; font-size:0.75rem; text-transform:uppercase; letter-spacing:0.08em;">
              Focus
            </div>
            <div style="margin-top:4px; font-weight:600;">Face swaps & edits</div>
            <div style="margin-top:2px; color:#9ca3af;">GAN, diffusion, classic fakes</div>
          </div>

          <div style="padding:10px 12px; border-radius:14px;
                      background:rgba(15,23,42,0.95); border:1px solid rgba(244,114,182,0.5); grid-column:span 2;">
            <div style="color:#f9a8d4; font-size:0.75rem; text-transform:uppercase; letter-spacing:0.08em;">
              Tip
            </div>
            <div style="margin-top:4px; color:#e5e7eb;">
              Upload a known real and a suspected fake in the <b>Compare</b> tab to see how their
              symmetry and texture signatures differ.
            </div>
          </div>
        </div>
      </div>
    </div>
  </div>
</div>
"""
                )

        # PAGE 2 – Upload (single image)
        with gr.Tab("Upload", id=1):
            with gr.Column(elem_classes="main-card"):
                gr.Markdown("📁 Upload Image", elem_classes="section-title")
                gr.Markdown(
                    "Choose a JPG or PNG image and preview it before analysis.",
                    elem_classes="section-subtitle",
                )
                with gr.Row():
                    with gr.Column():
                        upload_file = gr.File(
                            label="Choose Image",
                            file_types=[".png", ".jpg", ".jpeg"],
                            type="filepath",
                            elem_classes="upload-box",
                        )
                        upload_error = gr.Markdown(visible=False, elem_classes="error-box")
                        img_info = gr.Markdown("", visible=True)
                        analyse_btn = gr.Button(
                            "🔎 ANALYZE NOW",
                            elem_classes="pill-button",
                            interactive=False,
                        )
                        back_home1 = gr.Button("⬅ Back", elem_classes="secondary-pill")
                    with gr.Column():
                        preview = gr.Image(
                            label="Image Preview",
                            type="pil",
                            interactive=False,
                            visible=False,
                            elem_classes="preview-panel",
                        )

        # PAGE 3 – Compare two images (styled)
        with gr.Tab("Compare", id=2):
            with gr.Column(elem_classes="main-card"):
                gr.Markdown("🧬 Compare Two Images", elem_classes="section-title")
                gr.Markdown(
                    "Upload any two images to see how similar or different the deepfake signals are.",
                    elem_classes="section-subtitle",
                )

                with gr.Row():
                    with gr.Column():
                        gr.Markdown("### 🔹 Image A")
                        pair_a = gr.Image(type="pil", show_label=False)
                    with gr.Column():
                        gr.Markdown("### 🔸 Image B")
                        pair_b = gr.Image(type="pil", show_label=False)

                compare_error = gr.Markdown(visible=False, elem_classes="error-box")
                compare_btn = gr.Button("🔍 ANALYZE PAIR", elem_classes="pill-button")

                with gr.Row():
                    with gr.Column():
                        pair_summary = gr.Markdown(
                            visible=False,
                            elem_classes="details-box",
                            label="Pair summary",
                        )
                    with gr.Column():
                        pair_details = gr.Markdown(
                            visible=False,
                            elem_classes="details-box",
                            label="Per‑image details",
                        )

        # PAGE 4 – Result (single image)
        with gr.Tab("Result", id=3):
            with gr.Column(elem_classes="main-card"):
                gr.Markdown("🧪 Detection Result", elem_classes="section-title")
                gr.Markdown(
                    "See how the model interpreted this image.",
                    elem_classes="section-subtitle",
                )
                result_banner = gr.HTML()
                result_scores = gr.HTML()
                result_details = gr.Markdown(elem_classes="details-box")
                with gr.Row():
                    again_btn = gr.Button("🔁 Analyze Another", elem_classes="pill-button")
                    back_home2 = gr.Button("⬅ Back", elem_classes="secondary-pill")

        # PAGE 5 – About
        with gr.Tab("About", id=4):
            with gr.Column(elem_classes="main-card"):
                gr.Markdown("ℹ Model & Project Lab", elem_classes="section-title")
                gr.Markdown(
                    "Explore how this deepfake detector works under the hood.",
                    elem_classes="section-subtitle",
                )

                with gr.Row():
                    gr.HTML(
                        """
<div style="flex:1; padding:14px 16px; border-radius:16px;
            background:radial-gradient(circle at top,#1f2937,#020617);
            border:1px solid rgba(148,163,184,0.6); color:#e5e7eb; font-size:0.9rem;">
  <div style="font-size:0.8rem; text-transform:uppercase; letter-spacing:0.08em; color:#9ca3af;">
    Model core
  </div>
  <div style="font-size:1.05rem; font-weight:600; margin-top:4px;">
    DeiT-small Transformer
  </div>
  <div style="font-size:0.8rem; margin-top:6px; color:#9ca3af;">
    Lightweight vision transformer trained for image understanding.
  </div>
</div>
"""
                    )
                    gr.HTML(
                        """
<div style="flex:1; padding:14px 16px; border-radius:16px;
            background:radial-gradient(circle at top,#052e16,#020617);
            border:1px solid rgba(34,197,94,0.5); color:#e5e7eb; font-size:0.9rem;">
  <div style="font-size:0.8rem; text-transform:uppercase; letter-spacing:0.08em; color:#86efac;">
    Extra features
  </div>
  <div style="font-size:1.05rem; font-weight:600; margin-top:4px;">
    50‑D symmetry vector
  </div>
  <div style="font-size:0.8rem; margin-top:6px; color:#bbf7d0;">
    Summarises 68 facial landmarks into geometric imbalance scores.
  </div>
</div>
"""
                    )

                gr.HTML(
                    """
<div style="margin-top:22px; border-radius:18px;
            border:1px solid rgba(148,163,184,0.4); background:#020617; padding:16px 18px;">
  <details open>
    <summary style="cursor:pointer; font-weight:600; color:#e5e7eb;">
      🔧 Architecture walkthrough
    </summary>
    <div style="margin-top:10px; font-size:0.9rem; color:#e5e7eb;">
      <ol style="padding-left:18px; line-height:1.5; margin-top:6px;">
        <li><b>Backbone:</b> Image → DeiT-small vision transformer.</li>
        <li><b>Landmarks:</b> dlib predicts 68 keypoints → 50‑dim symmetry vector.</li>
        <li><b>Fusion:</b> CLS token + symmetry vector → single fused feature.</li>
        <li><b>Classifier:</b> Fully‑connected layer → REAL / FAKE logits.</li>
      </ol>
    </div>
  </details>
</div>
"""
                )

    # ---------- navigation & actions ----------
    start_btn.click(fn=lambda: go_tab(1), outputs=tabs)

    upload_file.upload(
        fn=handle_upload,
        inputs=upload_file,
        outputs=[upload_error, preview, analyse_btn, preview, img_info],
    )

    analyse_btn.click(
        fn=analyse_and_go,
        inputs=preview,
        outputs=[upload_error, result_banner, result_scores, result_details, tabs],
    )

    again_btn.click(fn=lambda: go_tab(1), outputs=tabs)
    back_home1.click(fn=lambda: go_tab(0), outputs=tabs)
    back_home2.click(fn=lambda: go_tab(0), outputs=tabs)

    compare_btn.click(
        fn=analyse_pair,
        inputs=[pair_a, pair_b],
        outputs=[compare_error, pair_summary, pair_details],
    )

print("\n🚀 Starting Gradio server…")
demo.launch(share=True, debug=False)


Device: cpu
Symmetry extractor ready. SYM_DIM = 50


Fast image processor class <class 'transformers.models.vit.image_processing_vit_fast.ViTImageProcessorFast'> is available for this model. Using slow image processor class. To use the fast image processor class set `use_fast=True`.


Loading DeiT backbone...


You are using a model of type vit to instantiate a model of type deit. This is not supported for all configurations of models and can yield errors.
Some weights of DeiTModel were not initialized from the model checkpoint at facebook/deit-small-patch16-224 and are newly initialized: ['embeddings.cls_token', 'embeddings.distillation_token', 'embeddings.patch_embeddings.projection.bias', 'embeddings.patch_embeddings.projection.weight', 'embeddings.position_embeddings', 'encoder.layer.0.attention.attention.key.bias', 'encoder.layer.0.attention.attention.key.weight', 'encoder.layer.0.attention.attention.query.bias', 'encoder.layer.0.attention.attention.query.weight', 'encoder.layer.0.attention.attention.value.bias', 'encoder.layer.0.attention.attention.value.weight', 'encoder.layer.0.attention.output.dense.bias', 'encoder.layer.0.attention.output.dense.weight', 'encoder.layer.0.intermediate.dense.bias', 'encoder.layer.0.intermediate.dense.weight', 'encoder.layer.0.layernorm_after.bias', 'en

✓ Trained model loaded from /content/drive/MyDrive/Deepfake/deit_fusion_epoch5.pth


/tmp/ipython-input-210091838.py:468: DeprecationWarning: The 'css' parameter in the Blocks constructor will be removed in Gradio 6.0. You will need to pass 'css' to Blocks.launch() instead.
  with gr.Blocks(title="Deepfake Detection", css=custom_css) as demo:



🚀 Starting Gradio server…
Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://517fb53e0ce95e856a.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
